![](image.jpg)


Dive into the heart of data science with a project that combines healthcare insights and predictive analytics. As a Data Scientist at a top Health Insurance company, you have the opportunity to predict customer healthcare costs using the power of machine learning. Your insights will help tailor services and guide customers in planning their healthcare expenses more effectively.

## Dataset Summary

Meet your primary tool: the `insurance.csv` dataset. Packed with information on health insurance customers, this dataset is your key to unlocking patterns in healthcare costs. Here's what you need to know about the data you'll be working with:

## insurance.csv
| Column    | Data Type | Description                                                      |
|-----------|-----------|------------------------------------------------------------------|
| `age`       | int       | Age of the primary beneficiary.                                  |
| `sex`       | object    | Gender of the insurance contractor (male or female).             |
| `bmi`       | float     | Body mass index, a key indicator of body fat based on height and weight. |
| `children`  | int       | Number of dependents covered by the insurance plan.              |
| `smoker`    | object    | Indicates whether the beneficiary smokes (yes or no).            |
| `region`    | object    | The beneficiary's residential area in the US, divided into four regions. |
| `charges`   | float     | Individual medical costs billed by health insurance.             |



A bit of data cleaning is key to ensure the dataset is ready for modeling. Once your model is built using the `insurance.csv` dataset, the next step is to apply it to the `validation_dataset.csv`. This new dataset, similar to your training data minus the `charges` column, tests your model's accuracy and real-world utility by predicting costs for new customers.

## Let's Get Started!

This project is your playground for applying data science in a meaningful way, offering insights that have real-world applications. Ready to explore the data and uncover insights that could revolutionize healthcare planning? Let's begin this exciting journey!

In [ ]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

# Loading the insurance dataset
insurance_data_path = 'insurance.csv'
insurance = pd.read_csv(insurance_data_path)
insurance.head()

In [ ]:
# Implement model creation and training here
# Use as many cells as you need

### Cleaning the Data

In [ ]:
def clean_insurance(df):
    insurance_cleaned = df.copy()
    insurance_cleaned = insurance_cleaned[insurance_cleaned['age'] > 0]
    insurance_cleaned['sex'] = (
        insurance_cleaned['sex']
        .astype('object')
        .map(lambda x: {'f': 'F', 'm': 'M'}.get(str(x).lower()[0], None))
    )
    insurance_cleaned['children'] = (
        insurance_cleaned['children']
        .clip(lower=0)  # Ensure no negative values
        .fillna(0)      # Replace NaN with 0
        .replace([float('inf'), -float('inf')], 0)  # Replace inf/-inf with 0
        .round(0)       # Round to nearest integer
        .astype('int32')  # Convert to int32
    )
    insurance_cleaned['smoker'] = (
        insurance_cleaned['smoker']
        .astype('object') 
        .map(lambda x: {'y': True, 'n': False}.get(str(x).lower()[0], None))
    )
    insurance_cleaned['region'] = insurance_cleaned['region'].str.lower()
    def clean_dollar_value(value):
        try:
            # Remove dollar sign and commas, and convert to float
            return float(str(value).replace('$', '').replace(',', '').strip())
        except ValueError:
            # Return NaN if conversion fails
            return float('nan')
    try:
        insurance_cleaned['charges'] = insurance_cleaned['charges'].map(clean_dollar_value)
    except:
        pass
    return insurance_cleaned

In [ ]:
insurance_cleaned = clean_insurance(insurance)
insurance_cleaned = insurance_cleaned.dropna()
insurance_cleaned.info()

In [ ]:
features_df = insurance_cleaned.drop('charges', axis=1)
features_dummified = pd.get_dummies(features_df, drop_first = True)
X = features_dummified.values
y = insurance_cleaned['charges'].values

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True)
model = LinearRegression()
pipeline = make_pipeline(StandardScaler(), model)
pipeline.fit(X_train, y_train)
r2_score = pipeline.score(X_test, y_test)
r2_score

In [ ]:
print(model.coef_, model.intercept_)

In [ ]:
validation_data_path = 'validation_dataset.csv'
validation_data = pd.read_csv(validation_data_path)

validation_cleaned = clean_insurance(validation_data)
validation_dummified = pd.get_dummies(validation_cleaned, drop_first = True)
X_validation = validation_dummified.values
predicted_charges = pipeline.predict(X_validation)
np.min(predicted_charges)

In [ ]:
validation_data['predicted_charges'] = predicted_charges.clip(1000)
np.min(validation_data['predicted_charges'])